# **Task 1: Exploratory Data Analysis and Data Preprocessing**
## CrediTrust Financial - Complaint Analysis System

This notebook performs comprehensive EDA on CFPB complaint data and preprocesses it for the RAG pipeline.

In [9]:
import sidetable as stb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.float_format', lambda x: '%.0f' % x)


In [10]:
# Load the dataset
data_path = Path("../data/raw/filtered_data.parquet")
if data_path.exists():
    df = pd.read_parquet(data_path)
    print(f"Loaded filtered dataset with {len(df)} records")
else:
    # Load from CSV if parquet doesn't exist
    df = pd.read_csv("../../data/raw/complaints.csv")
    print(f"Loaded raw dataset with {len(df)} records")

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

Loaded raw dataset with 9609797 records
Dataset shape: (9609797, 18)

Columns: ['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Company public response', 'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Date sent to company', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID']


In [11]:
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,Experian Information Solutions Inc.,FL,32092,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195687
1,2025-06-20,Debt collection,Telecommunications debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company can't verify or dispute the facts in t...,"Eastern Account Systems of Connecticut, Inc.",FL,342XX,NaN,NaN,Web,2025-06-20,Closed with explanation,Yes,NaN,14195688
2,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195689
3,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,Experian Information Solutions Inc.,AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195690
4,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,Experian Information Solutions Inc.,IL,60628,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9609792,2016-09-08,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting my free annual report,NaN,NaN,"EQUIFAX, INC.",FL,32853,NaN,Consent not provided,Web,2016-09-08,Closed with explanation,Yes,Yes,2102374
9609793,2022-06-06,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,"EQUIFAX, INC.",CA,93638,NaN,Consent not provided,Web,2022-06-06,Closed with explanation,Yes,NaN,5637858
9609794,2016-07-14,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting report or credit score,NaN,NaN,"EQUIFAX, INC.",NJ,08620,"Older American, Servicemember",NaN,Phone,2016-07-14,Closed with explanation,Yes,No,2012835
9609795,2014-03-03,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,NaN,NaN,"EQUIFAX, INC.",NJ,08759,NaN,NaN,Web,2014-03-04,Closed with explanation,Yes,No,740118


In [12]:
# Basic dataset information
print("=== DATASET OVERVIEW ===")
print(df.info())

# Identifying missing values
print("\n=== MISSING VALUES ===")
df.stb.missing()

=== DATASET OVERVIEW ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9609797 entries, 0 to 9609796
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    object
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              object
 16  Consumer disputed?            object
 17  Complaint ID                  int64 
dtypes: int64(1), obje

,missing,total,percent
Tags,8981029,9609797,93
Consumer disputed?,8841498,9609797,92
Consumer complaint narrative,6629041,9609797,69
Company public response,4770207,9609797,50
Consumer consent provided?,1649561,9609797,17
Sub-issue,839522,9609797,9
Sub-product,235295,9609797,2
State,54516,9609797,1
ZIP code,30228,9609797,0
Company response to consumer,20,9609797,0


In [13]:
# Target products for CrediTrust
target_products = [
    "Credit card",
    "Personal loan", 
    "Buy Now, Pay Later (BNPL)",
    "Savings account",
    "Money transfers"
]

# Analyze product distribution
print("=== PRODUCT DISTRIBUTION ===")
product_counts = df['Product'].value_counts()
print(product_counts.head(10))

# Check which target products exist in data
print("\n=== TARGET PRODUCTS AVAILABILITY ===")
available_products = []
for product in target_products:
    count = df[df['Product'].str.contains(product, case=False, na=False)]['Product'].value_counts()
    if len(count) > 0:
        print(f"{product}: {count.sum()} complaints")
        available_products.extend(count.index.tolist())
    else:
        print(f"{product}: 0 complaints")

=== PRODUCT DISTRIBUTION ===
Product
Credit reporting or other personal consumer reports                             4834855
Credit reporting, credit repair services, or other personal consumer reports    2163857
Debt collection                                                                  799197
Mortgage                                                                         422254
Checking or savings account                                                      291178
Credit card                                                                      226686
Credit card or prepaid card                                                      206369
Money transfer, virtual currency, or money service                               145066
Credit reporting                                                                 140429
Student loan                                                                     109717
Name: count, dtype: int64

=== TARGET PRODUCTS AVAILABILITY ===
Credit card: 433055

In [21]:
# Visualize product distribution
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Top 15 Products', 'Target Products Only', 'Complaints Over Time', 'Narrative Length Distribution'),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "scatter"}, {"type": "histogram"}]]
)

# Top products bar chart
top_products = product_counts.head(15)
fig.add_trace(
    go.Bar(x=top_products.values, y=top_products.index, orientation='h', name='All Products'),
    row=1, col=1
)

# Target products pie chart
target_df = df[df['Product'].isin(available_products)]
target_counts = target_df['Product'].value_counts()
fig.add_trace(
    go.Pie(labels=target_counts.index, values=target_counts.values, name='Target Products'),
    row=1, col=2
)

fig.update_layout(height=800, showlegend=False, title_text="Complaint Analysis Overview")
fig.show()

In [25]:
# Analyze complaint narratives
print("=== COMPLAINT NARRATIVE ANALYSIS ===")

# Check for narratives
narrative_col = 'Consumer complaint narrative' if 'Consumer complaint narrative' in df.columns else 'Complaint'
print(f"Using column: {narrative_col}")

# Count narratives
total_complaints = len(df)
with_narratives = df[narrative_col].notna().sum()
without_narratives = total_complaints - with_narratives

print(f"Total complaints: {total_complaints:,}")
print(f"With narratives: {with_narratives:,} ({with_narratives/total_complaints*100:.1f}%)")
print(f"Without narratives: {without_narratives:,} ({without_narratives/total_complaints*100:.1f}%)")

# Analyze narrative lengths
df['narrative_length'] = df[narrative_col].fillna('').astype(str).apply(len)
df['word_count'] = df[narrative_col].fillna('').astype(str).apply(lambda x: len(x.split()))

print(f"\n=== NARRATIVE LENGTH STATISTICS ===")
print(df['narrative_length'].describe())
print(f"\n=== WORD COUNT STATISTICS ===")
print(df['word_count'].describe())

=== COMPLAINT NARRATIVE ANALYSIS ===
Using column: Consumer complaint narrative
Total complaints: 9,609,797
With narratives: 2,980,756 (31.0%)
Without narratives: 6,629,041 (69.0%)

=== NARRATIVE LENGTH STATISTICS ===
count   9609797
mean        310
std         859
min           0
25%           0
50%           0
75%         282
max       35722
Name: narrative_length, dtype: float64

=== WORD COUNT STATISTICS ===
count   9609797
mean         54
std         150
min           0
25%           0
50%           0
75%          50
max        6469
Name: word_count, dtype: float64


In [26]:
# Text preprocessing function
def clean_narrative(text):
    """Clean complaint narrative text."""
    if pd.isna(text) or text == '':
        return ''
    
    text = str(text).lower()
    
    # Remove common boilerplate
    boilerplate_patterns = [
        r'i am writing to file a complaint',
        r'this is to inform you',
        r'dear sir/madam',
        r'to whom it may concern'
    ]
    
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Clean special characters but keep basic punctuation
    text = re.sub(r'[^a-zA-Z0-9\s.,!?-]', ' ', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply preprocessing
print("Cleaning narratives...")
df['cleaned_narrative'] = df[narrative_col].apply(clean_narrative)
df['cleaned_word_count'] = df['cleaned_narrative'].apply(lambda x: len(x.split()) if x else 0)

print("\n=== CLEANED NARRATIVE STATISTICS ===")
print(df['cleaned_word_count'].describe())

Cleaning narratives...

=== CLEANED NARRATIVE STATISTICS ===
count   9609797
mean         54
std         147
min           0
25%           0
50%           0
75%          50
max        6581
Name: cleaned_word_count, dtype: float64


In [27]:
# Filter dataset for target products and valid narratives
print("=== FILTERING DATASET ===")

# Filter for target products
filtered_df = df[df['Product'].isin(available_products)].copy()
print(f"After product filtering: {len(filtered_df)} records")

# Remove empty narratives
filtered_df = filtered_df[filtered_df['cleaned_narrative'].str.len() > 10].copy()
print(f"After removing empty narratives: {len(filtered_df)} records")

# Final dataset statistics
print(f"\n=== FINAL DATASET STATISTICS ===")
print(f"Total records: {len(filtered_df)}")
print(f"Products: {filtered_df['Product'].nunique()}")
print(f"Average narrative length: {filtered_df['cleaned_word_count'].mean():.1f} words")
print(f"Date range: {filtered_df['Date received'].min()} to {filtered_df['Date received'].max()}")

print(f"\n=== PRODUCT DISTRIBUTION IN FINAL DATASET ===")
print(filtered_df['Product'].value_counts())

=== FILTERING DATASET ===
After product filtering: 776742 records
After removing empty narratives: 357281 records

=== FINAL DATASET STATISTICS ===
Total records: 357281
Products: 6
Average narrative length: 216.7 words
Date range: 2015-03-19 to 2025-06-15

=== PRODUCT DISTRIBUTION IN FINAL DATASET ===
Product
Checking or savings account                                140316
Credit card or prepaid card                                108667
Credit card                                                 80667
Payday loan, title loan, or personal loan                   17238
Payday loan, title loan, personal loan, or advance loan      8896
Money transfers                                              1497
Name: count, dtype: int64


In [28]:
# Save filtered dataset
output_path = Path("../../data/processed/filtered_complaints.parquet")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Select relevant columns
final_columns = [
    'Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
    'Consumer complaint narrative', 'cleaned_narrative', 'Company', 'State'
]

# Keep only available columns
available_columns = [col for col in final_columns if col in filtered_df.columns]
final_df = filtered_df[available_columns].copy()

final_df.to_parquet(output_path, index=False)
print(f"Filtered dataset saved to: {output_path}")
print(f"Final dataset shape: {final_df.shape}")

Filtered dataset saved to: ..\..\data\processed\filtered_complaints.parquet
Final dataset shape: (357281, 9)


## Key Findings Summary

1. **Dataset Overview**: The dataset contains comprehensive complaint information with varying narrative lengths
2. **Product Coverage**: Successfully identified target products relevant to CrediTrust's services
3. **Data Quality**: Implemented cleaning procedures to remove boilerplate text and standardize narratives
4. **Final Dataset**: Created a clean, filtered dataset ready for embedding and RAG pipeline implementation